In [1]:
!pip install sqlalchemy
!pip install pyhive
!pip install thrift
!pip install sasl
!pip install thrift-sasl 
!pip install psycopg2

import psycopg2
import sqlalchemy
import pandas as pd
from sqlalchemy.engine import create_engine

In [2]:

engine = create_engine('hive://spark-thrift:10000/default')
pd.read_sql_query("create database if not exists sample",con=engine)
pd.read_sql_query("show databases",con=engine)


,namespace
0,default
1,sample


In [3]:
pd.read_sql_query("use sample", con=engine)
pd.read_sql_query("create table if not exists tbl1 (ckey int, cval varchar(64))", con=engine)
pd.read_sql_query("show tables", con=engine)


,database,tableName,isTemporary
0,sample,tbl1,False


In [4]:
pd.read_sql_query("insert into sample.tbl1 values (1, 'a');", con=engine)
pd.read_sql_query("select * from sample.tbl1;", con=engine)

,ckey,cval
0,1,a


In [5]:

pd.read_sql_query("show tables", con=engine)

,database,tableName,isTemporary
0,sample,tbl1,False


In [6]:
pd.read_sql_query("drop table if exists sample.hudi_tbl", con=engine);

In [7]:
pd.read_sql_query("""
    create table sample.hudi_tbl (
      id int,
      name string,
      price double,
      ts bigint
    ) 
    using hudi
    tblproperties (
      type = 'cow',
      primaryKey = 'id',
      hoodie.datasource.hive_sync.mode = 'JDBC'
    )
""", con=engine)

,Result


In [8]:
pd.read_sql_query("show tables", con=engine)

,database,tableName,isTemporary
0,sample,hudi_tbl,False
1,sample,tbl1,False


In [9]:
pd.read_sql_query("""

insert into sample.hudi_tbl 
values (1,'a',2.0, 1655951583.549022)

""", con=engine)

,Result


In [10]:
pd.read_sql_query("""select * from sample.hudi_tbl""", con=engine)

OperationalError: (pyhive.exc.OperationalError) TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=["*org.apache.hive.service.cli.HiveSQLException:Error running query: org.apache.spark.sql.AnalysisException: Table or view not found: ample.hudi_tbl; line 1 pos 14;\n'Project [*]\n+- 'UnresolvedRelation [ample, hudi_tbl]\n:37:36", 'org.apache.spark.sql.hive.thriftserver.SparkExecuteStatementOperation:org$apache$spark$sql$hive$thriftserver$SparkExecuteStatementOperation$$execute:SparkExecuteStatementOperation.scala:321', 'org.apache.spark.sql.hive.thriftserver.SparkExecuteStatementOperation:runInternal:SparkExecuteStatementOperation.scala:202', 'org.apache.hive.service.cli.operation.Operation:run:Operation.java:278', 'org.apache.spark.sql.hive.thriftserver.SparkExecuteStatementOperation:org$apache$spark$sql$hive$thriftserver$SparkOperation$$super$run:SparkExecuteStatementOperation.scala:46', 'org.apache.spark.sql.hive.thriftserver.SparkOperation:$anonfun$run$1:SparkOperation.s

In [ ]:
# using datetime module
import datetime;
  
# ct stores current time
ct = datetime.datetime.now()
print("current time:-", ct)
  
# ts store timestamp of current time
ts = ct.timestamp()
print("timestamp:-", ts)